UWAGA - poniżej również kod do testów na cpu. W celu lokalnych testów warto zrobić tymczasowy katalog z jednym datasetem, na podstawie którego będziemy porównywać czas. 

In [22]:
# path_to_datasets = './test_data/small/'
path_to_datasets = '../test_data/'

In [31]:
import cupy as cp

print("Czy GPU działa?", cp.cuda.is_available())
print("GPU:", cp.cuda.runtime.getDeviceProperties(0)['name'])

Czy GPU działa? True
GPU: b'NVIDIA GeForce RTX 4050 Laptop GPU'


# Training on gpu

UWAGA - możliwy tylko jak macie kartę zgodną z [CUDA](https://developer.nvidia.com/cuda-gpus). Później trzeba pobrać odpowiednie biblioteki zgodne z wersją cuda np. dla 11.2

```bash
pip install cupy-cuda112
pip install cuml-cu11
```
teoretycznie działa na google colabie, lokalnie idk, w razie problemów warto promptować

In [24]:
import os
import pandas as pd
import numpy as np
import cupy as cp
from cuml.preprocessing import StandardScaler
from cuml.ensemble import RandomForestRegressor
from cuml.model_selection import train_test_split
from cuml.metrics import mean_squared_error
import time
from datetime import timedelta

In [25]:
rf_100 =    ('RF', RandomForestRegressor(random_state=123, n_estimators=100))
rf_200 =    ('RF[200]', RandomForestRegressor(random_state=123, n_estimators=200))
rf_500 =    ('RF[500]', RandomForestRegressor(random_state=123, n_estimators=500))
rf_md_10 =  ('RF[md10]', RandomForestRegressor(random_state=123, max_depth=10))
rf_md_15 =  ('RF[md15]', RandomForestRegressor(random_state=123, max_depth=15))
rf_md_20 =  ('RF[md20]', RandomForestRegressor(random_state=123, max_depth=20))
rf_md_25 =  ('RF[md25]', RandomForestRegressor(random_state=123, max_depth=25))
rf_mss_3 =  ('RF[mss3]', RandomForestRegressor(random_state=123, min_samples_split=3))
rf_mss_4 =  ('RF[mss4]', RandomForestRegressor(random_state=123, min_samples_split=4))
rf_mss_6 =  ('RF[mss6]', RandomForestRegressor(random_state=123, min_samples_split=6))
rf_mss_8 =  ('RF[mss8]', RandomForestRegressor(random_state=123, min_samples_split=8))
rf_msl_2 =  ('RF[msl2]', RandomForestRegressor(random_state=123, min_samples_leaf=2))
rf_msl_3 =  ('RF[msl3]', RandomForestRegressor(random_state=123, min_samples_leaf=3))
rf_msl_4 =  ('RF[msl4]', RandomForestRegressor(random_state=123, min_samples_leaf=4))
rf_msl_5 =  ('RF[msl5]', RandomForestRegressor(random_state=123, min_samples_leaf=5))
rf_mf_log = ('RF[mfLog2]', RandomForestRegressor(random_state=123, max_features=0.301)) # log2 approximation
rf_mf_all = ('RF[mfAll]', RandomForestRegressor(random_state=123, max_features=1.0))

/home/bartekb/miniconda3/envs/rapids_env/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


In [26]:
max_samples = [0.2, 0.6, 0.8, 1, 1.2, 2, 3, 4, 5]
forests = [rf_100, rf_200, rf_500, rf_md_10, rf_md_15, rf_md_20, rf_md_25, rf_mss_3, rf_mss_4, rf_mss_6, rf_mss_8,
        rf_msl_2, rf_msl_3, rf_msl_4, rf_msl_5, rf_mf_log, rf_mf_all]

In [27]:
# Function to duplicate data for bootstrap rates > 1
def make_X_train(X_tra, y_tra, max_sample):
    if max_sample <= 1:
        return X_tra, y_tra
    elif 1 < max_sample < 2:
        # For GPU arrays, we need to use appropriate concatenation
        X_dup = cp.concatenate([X_tra, X_tra], axis=0)
        y_dup = cp.concatenate([y_tra, y_tra], axis=0)
        return X_dup, y_dup
    else:
        # Duplicate data max_sample times
        repeats = int(max_sample)
        X_list = [X_tra] * repeats
        y_list = [y_tra] * repeats
        X_dup = cp.concatenate(X_list, axis=0)
        y_dup = cp.concatenate(y_list, axis=0)
        return X_dup, y_dup

In [28]:

dataset_paths = []
for filename in os.listdir(path_to_datasets):
    full_path = os.path.join(path_to_datasets, filename)
    dataset_paths.append(full_path)

results = {'dataset': [],
           'bootstrap_rate': [],
           'rf': [],
           'cv_mse_scores': [],
           'mean_mse': [],
           'std_mse': []}

standard_scaler = StandardScaler()

n_splits = 2  # Original number of splits in KFold
n_repeats = 200
test_size = 1/n_splits  # 1/2 = 0.5 for test set in train_test_split
total_datasets = len(dataset_paths)
total_ms = len(max_samples)
total_forests = len(forests)
total_folds = n_splits * n_repeats

pd.read_csv(dataset_paths[0]).head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Sex_F,Sex_I,Sex_M,Rings
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,-0.674834,-0.688018,1.316677,15
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,-0.674834,-0.688018,1.316677,7
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,1.481846,-0.688018,-0.759488,9
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,-0.674834,-0.688018,1.316677,10
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,-0.674834,1.453451,-0.759488,7


In [ ]:
start_time = time.time()

print(f"Starting GPU-accelerated training with {total_datasets} datasets, {total_ms} bootstrap rates, {total_forests} forest configurations, and {total_folds} CV folds")

for dataset_idx, dataset_path in enumerate(dataset_paths, 1):
    dataset_name = os.path.basename(dataset_path)
    print(f"\nProcessing dataset {dataset_idx}/{total_datasets}: {dataset_name}")

    # Load data with pandas first
    df = pd.read_csv(dataset_path)

    # Convert to cuDF for GPU processing
    X_pd = df.iloc[:, :-1].values
    y_pd = df.iloc[:, -1].values

    # Convert to CuPy arrays for GPU
    X = cp.array(X_pd, dtype=cp.float32)
    y = cp.array(y_pd, dtype=cp.float32)

    # Standardize the features
    X = standard_scaler.fit_transform(X)

    dataset_start_time = time.time()

    for ms_idx, ms in enumerate(max_samples, 1):
        print(f"  Bootstrap rate {ms_idx}/{total_ms}: {ms}")

        for forest_idx, rfr in enumerate(forests, 1):
            forest_start_time = time.time()
            print(f"    Forest model {forest_idx}/{total_forests}: {rfr[0]}")
            cv_mse_scores = []

            # Simulate RepeatedKFold by using train_test_split multiple times with different random states
            for repeat in range(n_repeats):
                # For each repeat, we'll perform n_splits different splits (simulating KFold)
                for split in range(n_splits):
                    fold_num = repeat * n_splits + split + 1

                    if fold_num % 10 == 0:  # Print only every 10 folds to avoid too much output
                        print(f"      Processing fold {fold_num}/{total_folds}")

                    # Use different random state for each split
                    random_state = repeat * 100 + split

                    # Split data into train and test sets
                    X_train, X_test, y_train, y_test = train_test_split(
                        X, y, test_size=test_size, random_state=random_state
                    )

                    # Duplicate data based on bootstrap rate
                    X_tr, y_tr = make_X_train(X_train, y_train, ms)
                    # Set max_samples parameter for RandomForestRegressor
                    max_samples_value = None if ms >= 1 and ms != 1.2 else (0.6 if ms == 1.2 else ms)
                    rfr[1].max_samples = max_samples_value

                    # Fit the model
                    rfr[1].fit(X_tr, y_tr)

                    # Predict and evaluate
                    y_hat = rfr[1].predict(X_test)
                    mse = mean_squared_error(y_test, y_hat)
                    cv_mse_scores.append(float(mse))  # Convert from GPU to CPU

            # Record results
            results['dataset'].append(dataset_path)
            results['bootstrap_rate'].append(ms)
            results['rf'].append(rfr[0])
            results['cv_mse_scores'].append(cv_mse_scores)
            results['mean_mse'].append(f'{np.mean(cv_mse_scores):.3f}')
            results['std_mse'].append(f'{np.std(cv_mse_scores):.3f}')

            forest_time = time.time() - forest_start_time
            print(f"    Completed {rfr[0]} with bootstrap rate {ms}. Mean MSE: {np.mean(cv_mse_scores):.3f}")
            print(f"    Time taken: {timedelta(seconds=int(forest_time))}")

    dataset_time = time.time() - dataset_start_time
    print(f"  Dataset {dataset_name} completed in {timedelta(seconds=int(dataset_time))}")

total_time = time.time() - start_time
print(f"\nTraining complete in {timedelta(seconds=int(total_time))}. Saving results...")

wyniki = pd.DataFrame(results)
wyniki.to_csv('wyniki_bb.csv', index=False)
print("Results saved to 'wyniki_bb.csv'")

Starting GPU-accelerated training with 1 datasets, 9 bootstrap rates, 17 forest configurations, and 400 CV folds

Processing dataset 1/1: Abalone_1_preprocessed.csv


  Bootstrap rate 1/9: 0.2
    Forest model 1/17: RF
      Processing fold 10/400
      Processing fold 20/400


KeyboardInterrupt: 